In [15]:
import numpy as np
import pandas as pd

In [93]:
all_data = pd.DataFrame()
for w in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]:
    data = pd.read_csv(f'./data/pred/week_{w}.csv', compression='zip')
    all_data = pd.concat([all_data, data], ignore_index=True)

In [94]:
all_data.columns

Index(['gameId', 'playId', 'frameId', 'displayName', 'est_epa', 'preds',
       'optimal_receiver', 'epa_actual', 'optimal_epa', 'down', 'week',
       'team_name', 'team', 'possession', 'position', 'x', 'y', 'event'],
      dtype='object')

In [95]:
all_data.shape

(10755078, 18)

In [98]:
all_data.drop_duplicates(inplace=True)

In [135]:
all_data

,gameId,playId,frameId,displayName,est_epa,preds,optimal_receiver,epa_actual,optimal_epa,down,week,team_name,team,possession,position,x,y,event
0,2018090600,75,11,Malcolm Jenkins,NaN,NaN,NaN,NaN,NaN,NaN,1,PHI,home,0,SS,88.89,36.50,ball_snap
1,2018090600,75,11,Corey Graham,NaN,NaN,NaN,NaN,NaN,NaN,1,PHI,home,0,FS,86.47,22.03,ball_snap
2,2018090600,75,11,Rodney McLeod,NaN,NaN,NaN,NaN,NaN,NaN,1,PHI,home,0,FS,73.61,28.70,ball_snap
3,2018090600,75,11,Jordan Hicks,NaN,NaN,NaN,NaN,NaN,NaN,1,PHI,home,0,MLB,86.63,31.12,ball_snap
4,2018090600,75,11,Ronald Darby,NaN,NaN,NaN,NaN,NaN,NaN,1,PHI,home,0,CB,82.63,20.50,ball_snap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10755073,2018123011,4261,53,Darius Jennings,-0.47,0.650696,Anthony Firkser,-5.903777,-0.117232,2.0,17,TEN,home,1,WR,74.43,22.50,pass_outcome_interception
10755074,2018123011,4261,53,Taywan Taylor,-0.12,0.555446,Anthony Firkser,-5.903777,-0.117232,2.0,17,TEN,home,1,WR,68.90,48.65,pass_outcome_interception
10755075,2018123011,4261,53,Anthony Firkser,1.36,0.668875,Anthony Firkser,-5.903777,-0.117232,2.0,17,TEN,home,1,TE,64.43,23.20,pass_outcome_interception
10755076,2018123011,4261,53,Cameron Batson,-0.29,0.740729,Anthony Firkser,-5.903777,-0.117232,2.0,17,TEN,home,1,WR,71.33,1.38,pass_outcome_interception


In [173]:
grouped_data = all_data[all_data['position']=='QB'].groupby(['gameId', 'playId'], as_index=False).max()

In [174]:
mask = grouped_data.groupby(by=['displayName','playId'], as_index=False).count()['displayName'].value_counts()> 100

In [175]:
eligible_qb = grouped_data.groupby(by=['displayName','playId'], as_index=False).count()['displayName'].value_counts().index[mask]

In [176]:
grouped_data = all_data[(all_data['position']=='QB') & (all_data['displayName'].isin(eligible_qb))].groupby(['gameId', 'playId'], as_index=False).max()

In [177]:
grouped_data

,gameId,playId,frameId,displayName,est_epa,preds,optimal_receiver,epa_actual,optimal_epa,down,week,team_name,team,possession,position,x,y,event
0,2018090600,75,52,Matt Ryan,-0.35,0.494362,Julio Jones,0.261827,0.198432,1.0,1,ATL,away,1,QB,96.46,27.82,pass_outcome_caught
1,2018090600,146,48,Matt Ryan,-0.46,0.692313,Marvin Hall,-0.372360,-0.006367,1.0,1,ATL,away,1,QB,54.77,29.91,pass_outcome_incomplete
2,2018090600,168,47,Matt Ryan,-1.14,0.705297,Austin Hooper,-0.702779,-0.200630,2.0,1,ATL,away,1,QB,55.38,29.77,pass_outcome_incomplete
3,2018090600,190,62,Matt Ryan,-1.43,0.685692,Julio Jones,3.047530,1.341028,3.0,1,ATL,away,1,QB,58.11,29.61,pass_outcome_caught
4,2018090600,256,46,Matt Ryan,-1.47,0.629862,Devonta Freeman,-0.842272,1.721178,3.0,1,ATL,away,1,QB,18.59,25.33,pass_outcome_incomplete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16326,2018123015,3794,54,Josh Rosen,-0.37,0.503461,Trent Sherfield,1.031088,0.112559,1.0,17,ARI,away,1,QB,91.84,27.28,play_action
16327,2018123015,3819,52,Josh Rosen,-0.42,0.621271,Chad Williams,0.732852,0.645166,1.0,17,ARI,away,1,QB,74.37,26.76,play_action
16328,2018123015,3969,60,Josh Rosen,-1.38,0.707269,Trent Sherfield,-0.944168,0.604044,3.0,17,ARI,away,1,QB,55.32,25.73,pass_outcome_incomplete
16329,2018123015,4057,47,Russell Wilson,-3.55,0.584226,Ed Dickson,-0.254137,-1.079540,2.0,17,SEA,home,1,QB,90.51,29.29,pass_outcome_caught


In [178]:
def qbiq(x):
    if x['epa_actual'] > x['optimal_epa']:
        return 1
    else:
         return 0

In [179]:
def qbiq2(x):
    if x['epa_actual'] > x['optimal_epa']:
        return 1
    
    if x['epa_actual'] < x['optimal_epa'] and x['epa_actual'] > 0:
        return .5
    
    else:
         return 0

In [180]:
grouped_data['qbiq'] = grouped_data.apply(qbiq, axis=1)
grouped_data['qbiq2'] = grouped_data.apply(qbiq2, axis=1)
grouped_data['qbiq3'] = grouped_data['epa_actual'] - grouped_data['optimal_epa']

In [181]:
grouped_data

,gameId,playId,frameId,displayName,est_epa,preds,optimal_receiver,epa_actual,optimal_epa,down,...,team_name,team,possession,position,x,y,event,qbiq,qbiq2,qbiq3
0,2018090600,75,52,Matt Ryan,-0.35,0.494362,Julio Jones,0.261827,0.198432,1.0,...,ATL,away,1,QB,96.46,27.82,pass_outcome_caught,1,1.0,0.063395
1,2018090600,146,48,Matt Ryan,-0.46,0.692313,Marvin Hall,-0.372360,-0.006367,1.0,...,ATL,away,1,QB,54.77,29.91,pass_outcome_incomplete,0,0.0,-0.365993
2,2018090600,168,47,Matt Ryan,-1.14,0.705297,Austin Hooper,-0.702779,-0.200630,2.0,...,ATL,away,1,QB,55.38,29.77,pass_outcome_incomplete,0,0.0,-0.502149
3,2018090600,190,62,Matt Ryan,-1.43,0.685692,Julio Jones,3.047530,1.341028,3.0,...,ATL,away,1,QB,58.11,29.61,pass_outcome_caught,1,1.0,1.706502
4,2018090600,256,46,Matt Ryan,-1.47,0.629862,Devonta Freeman,-0.842272,1.721178,3.0,...,ATL,away,1,QB,18.59,25.33,pass_outcome_incomplete,0,0.0,-2.563450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16326,2018123015,3794,54,Josh Rosen,-0.37,0.503461,Trent Sherfield,1.031088,0.112559,1.0,...,ARI,away,1,QB,91.84,27.28,play_action,1,1.0,0.918529
16327,2018123015,3819,52,Josh Rosen,-0.42,0.621271,Chad Williams,0.732852,0.645166,1.0,...,ARI,away,1,QB,74.37,26.76,play_action,1,1.0,0.087686
16328,2018123015,3969,60,Josh Rosen,-1.38,0.707269,Trent Sherfield,-0.944168,0.604044,3.0,...,ARI,away,1,QB,55.32,25.73,pass_outcome_incomplete,0,0.0,-1.548212
16329,2018123015,4057,47,Russell Wilson,-3.55,0.584226,Ed Dickson,-0.254137,-1.079540,2.0,...,SEA,home,1,QB,90.51,29.29,pass_outcome_caught,1,1.0,0.825403


In [182]:
grouped_data.groupby(by=['displayName']).mean().sort_values(by=['qbiq2'], ascending=False)[['epa_actual', 'optimal_epa', 'qbiq', 'qbiq2', 'qbiq3']]

,epa_actual,optimal_epa,qbiq,qbiq2,qbiq3
displayName,,,,,
Philip Rivers,0.292080,0.483545,0.493274,0.562780,-0.191465
Matt Ryan,0.316536,0.460510,0.491694,0.553987,-0.143974
Andrew Luck,0.216435,0.399232,0.492188,0.550781,-0.182797
Mitchell Trubisky,0.217458,0.448662,0.485507,0.538647,-0.231204
Deshaun Watson,0.292898,0.633256,0.473896,0.537149,-0.340358
Ryan Fitzpatrick,0.285341,0.451595,0.487705,0.536885,-0.166254
Carson Wentz,0.243233,0.534952,0.475000,0.536250,-0.291719
Nick Mullens,0.243727,0.660046,0.459854,0.532847,-0.416319
Drew Brees,0.362536,0.676816,0.461538,0.532389,-0.314280


In [183]:
grouped_data.groupby(by=['displayName']).mean().sort_values(by=['qbiq'], ascending=False)[['epa_actual', 'optimal_epa', 'qbiq', 'qbiq2', 'qbiq3']]

,epa_actual,optimal_epa,qbiq,qbiq2,qbiq3
displayName,,,,,
Philip Rivers,0.292080,0.483545,0.493274,0.562780,-0.191465
Andrew Luck,0.216435,0.399232,0.492188,0.550781,-0.182797
Matt Ryan,0.316536,0.460510,0.491694,0.553987,-0.143974
Ryan Fitzpatrick,0.285341,0.451595,0.487705,0.536885,-0.166254
Mitchell Trubisky,0.217458,0.448662,0.485507,0.538647,-0.231204
Carson Wentz,0.243233,0.534952,0.475000,0.536250,-0.291719
Deshaun Watson,0.292898,0.633256,0.473896,0.537149,-0.340358
Aaron Rodgers,0.216398,0.485034,0.465662,0.522613,-0.268636
Andy Dalton,0.182018,0.346947,0.465565,0.523416,-0.164928


In [184]:
grouped_data.groupby(by=['displayName']).mean().sort_values(by=['qbiq3'], ascending=False)[['epa_actual', 'optimal_epa', 'qbiq', 'qbiq2', 'qbiq3']]

,epa_actual,optimal_epa,qbiq,qbiq2,qbiq3
displayName,,,,,
Matt Ryan,0.316536,0.460510,0.491694,0.553987,-0.143974
Andy Dalton,0.182018,0.346947,0.465565,0.523416,-0.164928
Ryan Fitzpatrick,0.285341,0.451595,0.487705,0.536885,-0.166254
Matthew Stafford,0.168873,0.341510,0.445438,0.500000,-0.172637
Andrew Luck,0.216435,0.399232,0.492188,0.550781,-0.182797
Cody Kessler,-0.003783,0.180844,0.458647,0.496241,-0.184627
Philip Rivers,0.292080,0.483545,0.493274,0.562780,-0.191465
Patrick Mahomes,0.435681,0.628310,0.450368,0.522059,-0.192629
Ryan Tannehill,0.068118,0.298628,0.455882,0.500000,-0.230510


In [186]:
qbiq_data = grouped_data.groupby(by=['displayName']).mean().sort_values(by=['qbiq'], ascending=False)[['epa_actual', 'optimal_epa', 'qbiq']]

In [187]:
qbiq_data['adjQBiQ'] = (1 + qbiq_data['qbiq']) * qbiq_data['epa_actual']

In [192]:
qbiq_data.sort_values(by=['adjQBiQ'], ascending=False)

,epa_actual,optimal_epa,qbiq,adjQBiQ
displayName,,,,
Patrick Mahomes,0.435681,0.628310,0.450368,0.631897
Drew Brees,0.362536,0.676816,0.461538,0.529860
Jared Goff,0.347175,0.776572,0.448649,0.502935
Russell Wilson,0.336921,0.747694,0.426357,0.480569
Matt Ryan,0.316536,0.460510,0.491694,0.472175
Philip Rivers,0.292080,0.483545,0.493274,0.436155
Deshaun Watson,0.292898,0.633256,0.473896,0.431701
Ryan Fitzpatrick,0.285341,0.451595,0.487705,0.424504
Carson Wentz,0.243233,0.534952,0.475000,0.358769
